In [3]:
import numpy as np
import matplotlib.pyplot as plt

## SSerr
求解最小误差平方和的最佳解（Linear Regression Algorithm），并完成二元分类。
- 函数的输入有：   
    （a）一个N*d维的矩阵X，它的第i行是第i个数据向量；    
    （b）一个N维列向量y，y的第i个元素包含了类（-1，1），并且该类与相应的向量相互对应；   
- 函数的输出为：向量w，返回估计参数向量。


In [4]:
def SSerr(X,y):
    Xt=np.transpose(X)
    Xp=np.linalg.pinv(Xt)
    yt=np.transpose(y)
    return np.dot(yt,Xp)

In [5]:
def SSerr2(X,y):
    Xt=np.transpose(X)
    Xp=np.dot(np.linalg.inv(np.dot(Xt,X)),Xt)
    #Xp=np.linalg.pinv(Xt)
    yt=np.transpose(y)
    return np.dot(Xp,y)

In [6]:
def gradientE(X,y,w):
    Xt=np.transpose(X)
    return (np.dot(np.dot(Xt,X),w)-np.dot(Xt,y))


In [7]:
def SSerr3(X,y):
    y=y.reshape([X.shape[0],1])
    # init w
    w=np.zeros([X.shape[1],1])
    # interate
    learning_rate=0.1
    epoch=1000
    for i in range(epoch):
        gradient=gradientE(X,y,w)
        w=w-learning_rate*gradient
        if i>epoch-20:
            print(w)
    #np.dot(X,w)

In [8]:
SSerr3(Xnp,Ynp)

NameError: name 'Xnp' is not defined

In [ ]:

dirtyData=[(0.2,0.7),1,
(0.3,0.3),1,
(0.4,0.5),1,
(0.6,0.5),1,
(0.1,0.4),1,
(0.4,0.6),-1,
(0.6,0.2),-1,
(0.7,0.4),-1,
(0.8,0.6),-1,
(0.7,0.5),-1]
X=[]
Y=[]
i=0
for data in dirtyData:
    #print(data)
    if i%2==0:
        X.append(data)
    else:
        Y.append(data)
    i+=1

In [ ]:
print(SSerr(X,Y))

In [ ]:
Xnp=np.asarray(X)
Ynp=np.asarray(Y)
print(Xnp.shape)
print(Ynp.shape)

In [ ]:
print(SSerr2(Xnp,Ynp))

## LMS
编写一个名为LMSalg的函数，用它来实现Widrow-Hoff算法（LMS算法），即用随机梯度下降法获得最佳解。
函数的输入/输出有：
- （a）一个N*d维的矩阵X，它的第i行是第i个数据向量；
- （b）一个N维列向量y，y的第i个元素包含了类（-1，1），并且该类与相应的向量相互对应；
- （c）用向量w_ini初始化参数向量；并且返回估计参数向量。


### addBias
为维数为[N,d]的向量X添加偏置向量

In [15]:
def addBias(X):
    N=X.shape[0]
    d=X.shape[1]
    bias_vector=np.ones([N,1])
    return np.column_stack((bias_vector,X))

In [16]:
addBias(np.zeros([2,2]))

array([[1., 0., 0.],
       [1., 0., 0.]])